In [5]:
import ast
import inspect

In [6]:
class FunctionAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.function_calls = []
        self.variables = []
        self.assignments = []

    def visit_Call(self, node):
        if isinstance(node.func, ast.Name):
            self.function_calls.append(node.func.id)
        self.generic_visit(node)

    def visit_Name(self, node):
        if isinstance(node.ctx, ast.Store):
            self.variables.append(node.id)
        self.generic_visit(node)

    def visit_Assign(self, node):
        for target in node.targets:
            if isinstance(target, ast.Name):
                self.assignments.append((target.id, ast.unparse(node.value)))
        self.generic_visit(node)

In [7]:
def parse_function(func):
    """
    Parses a given function using the ast module.
    
    :param func: The function to parse
    :return: A dictionary containing information about the function
    """
    # Get the source code of the function
    source = inspect.getsource(func)
    
    # Parse the source code into an AST
    tree = ast.parse(source)
    
    # Find the function definition node
    function_def = next((node for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)), None)
    
    if function_def is None:
        raise ValueError("Could not find function definition in the provided source")
    
    # Analyze the function
    analyzer = FunctionAnalyzer()
    analyzer.visit(function_def)
    
    # Extract information
    args = [arg.arg for arg in function_def.args.args]
    docstring = ast.get_docstring(function_def)
    
    return {
        "name": function_def.name,
        "args": args,
        "docstring": docstring,
        "function_calls": analyzer.function_calls,
        "variables": analyzer.variables,
        "assignments": analyzer.assignments,
    }

In [8]:
# Example usage
def example_function(a, b):
    """This is an example function."""
    x = a + b
    y = [i for i in range(10)]
    i = 7
    return sum(y) + x

result = parse_function(example_function)

print("Function Analysis Result:")
for key, value in result.items():
    print(f"{key}: {value}")

Function Analysis Result:
name: example_function
args: ['a', 'b']
docstring: This is an example function.
function_calls: ['range', 'sum']
variables: ['x', 'y', 'i', 'i']
assignments: [('x', 'a + b'), ('y', '[i for i in range(10)]'), ('i', '7')]
